In [1]:
import pandas as pd
import numpy as np

In [2]:
import spacy

from spacy.lang.en import English
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

from spacy.tokens import Doc
# import en_core_web_lg
# nlp = spacy.load('../models/spacy_multi_cat_model/')
nlp = spacy.load("../models/spacy_2/")


In [3]:
import sys

# add src folder to path
sys.path.insert(1, '../src')

# from text_prep import tidy_series, uppercase_proportion_column
from spacy_helper import doc_check

In [4]:
one_text = '''
Ok, let me say it again Come on, now you guys are just being piece of shit jews. I mean you have to admit, the guys in pink floyd play their instruments about as slow as a nigger works. I shouldn't even call what they play music. It's just a bunch of alarm clocks and cashier regirsters! But you know what the most pretentious thing about them is, its their lyrics. All af their songs are just surrealist poetry sung over doom-noise pop, and everyone starts calling them genius's over it. The truth is, their songs have no meaning. Take the album 'The Wall' for instance; sure it tells a story, but what is the moral and the meaning of the story? And dont tell me that the purpose of their songs is to make you think. The only way that music as slow as pink floyd could make you fucking think is if you were just as stoned as they are, which you kikes probly are... And one last time: 1) pink floyd fucking sucks 2) david fuckmor should taste my ass 3) you should to
'''

In [5]:
doc=nlp(one_text)
doc.cats

{'TOXIC': 0.9894261360168457,
 'SEVERE_TOXIC': 0.007514932658523321,
 'OBSCENE': 0.9430854916572571,
 'THREAT': 0.007477084174752235,
 'INSULT': 0.917741596698761,
 'IDENTITY_HATE': 0.7508668899536133}

## Before 5 epochs  
{'TOXIC': 0.9983283877372742,  
 'SEVERE_TOXIC': 0.0003475894336588681,  
 'OBSCENE': 0.2968767285346985,  
 'THREAT': 0.0011758265318349004,  
 'INSULT': 0.20628367364406586,  
 'IDENTITY_HATE': 0.00019311188952997327}  


## After 5 epochs  
{'TOXIC': 0.9834251999855042,  
 'SEVERE_TOXIC': 0.07075302302837372,  
 'OBSCENE': 0.8929150104522705,  
 'THREAT': 0.0113288052380085,  
 'INSULT': 0.8754300475120544,  
 'IDENTITY_HATE': 0.3847048282623291}  

## After 25 epochs  
  
{'TOXIC': 0.9894261360168457,  
 'SEVERE_TOXIC': 0.007514932658523321,  
 'OBSCENE': 0.9430854916572571,  
 'THREAT': 0.007477084174752235,  
 'INSULT': 0.917741596698761,  
 'IDENTITY_HATE': 0.7508668899536133}  

# Getting info from preserved spaCy docs

I've had a little difficulty with getting the doc properties to un-pickle and maintain the ability to further process them later. docs seem to depend on some vocab properties of the model that are not saved within the doc itself.

In [6]:
%%time
'''
CPU times: user 3min 44s, sys: 33.5 s, total: 4min 17s
Wall time: 4min 32s
'''
X_train = pd.read_pickle('../data/basic_df_split/X_train_2-1.pkl')

CPU times: user 3min 38s, sys: 28.4 s, total: 4min 6s
Wall time: 4min 17s


In [7]:
# load y_train
! ls ../data/basic_df_split/
y_train = pd.read_pickle('../data/basic_df_split/basic_y_train.pkl')

X_train_2-1.pkl         basic_X_test.pkl        basic_y_test.pkl
X_train_docs_series.pkl basic_X_train.pkl       basic_y_train.pkl


In [8]:
# test
tiny_doc_sample = X_train['docs'].head(2)
tiny_doc_sample

27301     (', Meša, Selimović, I, 'm, not, opposing, suc...
141668    (', September, 2008, (, UTC, ), Talking, about...
Name: docs, dtype: object

In [9]:
tiny_lemmas = tiny_doc_sample.apply(lambda doc: [nlp.vocab[tok.lemma].vector for tok in doc if doc_check(tok) and tok.has_vector])

In [10]:
len(tiny_lemmas[27301])

27

In [11]:
X_train.columns

Index(['comment_text', 'uppercase_proportion', 'docs', 'lemmas', 'doc_vectors',
       'tok_vectors'],
      dtype='object')

In [12]:
#
from sklearn.naive_bayes import MultinomialNB, BernoulliNB


In [13]:
# print(y_train['toxic'].head())
print(X_train['doc_vectors'])

27301     [-0.019261397, 0.1402782, -0.17225473, -0.0019...
141668    [-0.14023165, 0.22500116, -0.07992911, -0.0451...
104554    [0.0006661963, 0.2898478, -0.053957734, 0.0774...
118561    [-0.034046583, 0.17167076, -0.03414113, -0.051...
100409    [-0.07457596, 0.19859332, -0.21044752, -0.0248...
                                ...                        
153719    [-0.053927306, 0.14099398, -0.12922487, -0.099...
148645    [-0.04553457, 0.20419797, -0.14104365, -0.0279...
113159    [-0.04607839, 0.13241082, -0.17935269, -0.0352...
104771    [-0.24597338, 0.28444514, -0.07856212, -0.1289...
14596     [-0.12666084, 0.06894974, -0.2519976, -0.13606...
Name: doc_vectors, Length: 106912, dtype: object


# Where I Left Off

https://medium.com/@awantikdas/a-comprehensive-naive-bayes-tutorial-using-scikit-learn-f6b71ae84431

In [14]:
# nb_toxic_yn_doc_vect = BernoulliNB()

# nb_toxic_yn_doc_vect.fit(X_train['doc_vectors'], y_train['toxic'])

In [39]:
import matplotlib.pyplot as plt

# from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics

### Create list of lemmas

In [16]:
%%time
lemma_words = list()

for lst in X_train['lemmas']:
    for word in lst:
        lemma_words.append(word)


CPU times: user 538 ms, sys: 103 ms, total: 641 ms
Wall time: 649 ms


### Counter for lemmas

In [17]:
from collections import Counter

In [18]:
lemma_counter = Counter(lemma_words)

In [19]:
# 3151394 total lemmas in lemma_words
print(len(lemma_words))

print(lemma_counter.most_common(10))

# 116496 unique lemmas
print(len(lemma_counter))

3151394
[('article', 49387), ('page', 38252), ('wikipedia', 30307), ('talk', 25655), ('edit', 23369), ('like', 19462), ('think', 16678), ('know', 16297), ('source', 15575), ('thank', 15039)]
116496


In [20]:
# nltk imports
import nltk
from nltk.corpus import stopwords

# from nltk.stem import WordNetLemmatizer

In [21]:
# %%time
'''
CPU times: user 246 ms, sys: 21.2 ms, total: 268 ms
Wall time: 276 ms
'''
print(type(stopwords.words('english')))

stopw_set = set(stopwords.words('english'))

# lemma_less = [lemma for lemma in lemma_words if lemma not in stopw_set]

<class 'list'>


In [22]:
'''
lemma_less length:    3135975
reduced lemmas by:      15419
'''

# print(f'lemma_less length: {len(lemma_less):10}')
# print(f'reduced lemmas by: {len(lemma_words) - len(lemma_less):10}')

'\nlemma_less length:    3135975\nreduced lemmas by:      15419\n'

In [23]:
'''
lemma_less_counter length:     116427
reduced lemma_counter by:          69
'''
# lemma_less_counter = Counter(lemma_less)
# print(f'lemma_less_counter length: {len(lemma_less_counter):10}')
# print(f'reduced lemma_counter by:  {len(lemma_counter) - len(lemma_less_counter):10}')

'\nlemma_less_counter length:     116427\nreduced lemma_counter by:          69\n'

# TF*IDF

Working through techniques from lecture

In [24]:
X_train['lemmas'].head()

27301     [meša, selimović, oppose, formulation, instead...
141668    [september, utc, talk, victimize, release, imp...
104554    [august, utc, danke, support, find, need, ask,...
118561    [aurora, part, story, particularly, weak, auro...
100409    [wp, w, wikipedia, manual, text, give, instruc...
Name: lemmas, dtype: object

In [25]:
%%time
# remove lemmas that appear in nltk stopword list
X_train['lemmas_less'] = X_train['lemmas'].apply(lambda row: [lemma for lemma in row if lemma not in stopw_set])

CPU times: user 985 ms, sys: 507 ms, total: 1.49 s
Wall time: 1.8 s


## TF

In [26]:
def tf(processed_doc):
    '''
    from lecture.
    
    calculates the term frequency for every word in a document
    
    processed_doc is a list of words that have already had processing applied
    '''
    # word counts from Counter
    counts = Counter(processed_doc).most_common()
    
    # total_words is just the sum of all the counts in our counter
    total_words = sum([count for word, count in counts])
    
    # tf is a tuple of the word and the count for that word / total word count.
    tf = [(word, count/total_words) for word, count in counts]
    return dict(tf)

In [27]:
%%time
# create column with tf information per document
X_train['tf'] = X_train['lemmas_less'].apply(tf)

CPU times: user 1.92 s, sys: 1.03 s, total: 2.95 s
Wall time: 5.43 s


In [28]:
# peek at values
print(X_train['tf'].iloc[0])
print(sum(X_train['tf'].iloc[0].values()))

{'writer': 0.1, 'try': 0.06666666666666667, 'view': 0.06666666666666667, 'bosnian': 0.06666666666666667, 'serbian': 0.06666666666666667, 'yugoslavian': 0.06666666666666667, 'meša': 0.03333333333333333, 'selimović': 0.03333333333333333, 'oppose': 0.03333333333333333, 'formulation': 0.03333333333333333, 'instead': 0.03333333333333333, 'prevent': 0.03333333333333333, 'potential': 0.03333333333333333, 'editwar': 0.03333333333333333, 'inclusion': 0.03333333333333333, 'option': 0.03333333333333333, 'mention': 0.03333333333333333, 'read': 0.03333333333333333, 'cumbersome': 0.03333333333333333, 'maybe': 0.03333333333333333, 'worth': 0.03333333333333333, 'good': 0.03333333333333333, 'talk': 0.03333333333333333}
0.9999999999999999


## IDF

In [29]:
def get_vocabulary_from_s(series):
    '''
    get vocabulary from series of lists of lemmas
    '''
    # get unique set of tokens
    s = series.apply(lambda x: set(x))
    
    vocabulary = set()
    
    s.apply(lambda x: vocabulary.update(x))
    
    return vocabulary

In [30]:
def idf(corpus, vocabulary=None):
    '''
    calculates the inverse document frequency
    '''
    
    # get the vocabulary if None
    if vocabulary == None:
        vocabulary = get_vocabulary(corpus)

    # the top portion of the fraction in the log
    number_of_docs = len(corpus)

    docs = [set(doc) for doc in corpus]
    
    # create a list to store the tuples of ("word", idfvalue)
    idf_values = []
    # iterate through all words in the vocabulary to calculate idf values
    for word in vocabulary:
        # [doc for doc in docs if word in doc] is list of documents containing the word.
        docs_which_contain_the_word = [doc for doc in docs if word in doc]
        number_of_docs_with_word = len(docs_which_contain_the_word)
        # append the idf value to our list
        idf_values.append((word, np.log(number_of_docs / number_of_docs_with_word)))
    # return as a dictionary
    return dict(idf_values)

In [31]:
%%time

# Get training corpus vocabulary
X_train_vocab = get_vocabulary_from_s(X_train['lemmas_less'])

CPU times: user 1.39 s, sys: 836 ms, total: 2.23 s
Wall time: 4.15 s


In [32]:
print(f'X_train_vocab is a set with {len(X_train_vocab)} values')

X_train_vocab is a set with 116427 values


In [50]:
print([i for i in X_train_vocab if len(i) <= 2])


['nb', 'ŀ', 'ö', 'ef', 'ʎ', 'xz', '牛岩', 'gz', 'ps', 'î', 'ν', 'вы', 'mh', 'gq', 'ai', 'қ', 'si', 'dh', 'ʊ', 'ვ', 'r', 'ϸ', 'bz', 'ʡ', '雲水', 'ɢ', 'pr', 'bt', 'iw', '𐌲', 'въ', 'من', 'π', 'ή', 'ō', 'ŷ', 'þ', 'wp', 'jp', 'se', 'μm', 'sq', 'чи', 'ey', 'kg', 'œ', '濤', 'aç', '討論', 'ns', 'ed', 'za', 'ac', '咨', 'bs', 'cs', 'cứ', 'dm', 'ĭ', 'ħ', 'rt', 'θ', 'ń', 'ü', 'כל', 'ur', 'æ', 'sr', 'iy', 'tn', 'ě', 'eø', 'tz', 'ŗ', '対馬', '竹島', 'º', 'ʒ', 'im', 'lj', 'hb', 'ő', 'mg', 'fs', 'ǫ', 'fv', 'ny', '𒁳', '青い', 'fy', 'wk', 'р', 'λ', 'gf', '先聖', 'ŋ', 'χo', 'ʲ', 'xi', 'ha', 'τη', 'uj', 'yy', 'sy', 'lr', 'ā', 'مر', 'dd', 'sc', 'rw', 'זו', 'el', 'ṛ', 'fr', 'sb', 'ˠ', 'uo', 'sẽ', 'tự', '粵語', 'ws', 'ム', 'wa', 'ó', 'hg', 'ku', 'et', 'جا', 'uy', 'ʂ', 'fk', 'би', 'ш', 'qc', 'ar', 'av', 'nx', 'tq', '姓', 'cd', 'ℳ', 'ճ', 'я', 'œf', 'jg', 'fn', 'vr', 'ɜ', '縣', 'tv', 'hv', 'חג', 'gs', 'ck', 'yk', 'qs', 'ax', 'ĵ', '卐', 'ÿ', 'ɻ', 'ju', 'jə', 'yd', 'ig', 'tm', 'wr', 'hi', 'tk', 'je', 'v', 'kv', 'мы', '盒', 'es', 'pc', 

In [55]:
print([i for i in X_train_vocab if len(i) > 20])

['deletionaddictionteam', 'appreciatedespecially', 'pseudoantidisestablishmentarianism', 'kennnnnnnnnnnnnnnnnnnnnnnnnnnnnnedddddddddddddddddddddyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy', 'lololololololololpenis', 'peeeeeeeeeeniiiiiiiiiiiiisssss', 'mattcryptotabsnoswpinner', 'couldyoufixsevillaplease', 'moooovvvvvveeeeeeeeeerrrrrrrrr', 'woooooooooooooooooooo', 'titlewithoutnamespace', 'nooooooooooooooooooooooooooooootable', 'blahhhhhhhhhhhblahhhhhhhhhblahblahblah', 'peoplesfreesaveliytalk', 'armchairvexillogistdon', 'sooooooooooooooooooooo', 'hahahahahahahahahahahahaahhahahahahaahhahahahahahahaha', 'vertebralcompressionfractures', 'honorificabilitudinitatibus', 'gothaparduskerialldrapolatkh', 'superflyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy', 'consensusnotwithstande', 'umsomekindofendinghere', 'weburiedoursecretsinthegarden', 'revolvingpersonalityconduct', 'drivesfastturnsleftandright', 'douuuchhhheeeebaggggggg', 'ssssssssssssshh

In [33]:
# %%time
# '''
# CPU times: user 18min 49s, sys: 3.85 s, total: 18min 53s
# Wall time: 18min 57s
# '''
# # get idf dict
# idf_dict = idf(X_train['lemmas_less'].to_list(), vocabulary=X_train_vocab)

## Further reduce lemmas by min_length & max_length

Exploration of corpus vocabulary suggests that lemmas of 2 or fewer characters are likely not very useful and can be removed to reduce features. 

Lemmas of longer than 20 characters are often run-on words (where spaces have been omitted). Although a few of them have words hidden within them that may be considered toxic, the rarity and non-standard format make them unlikely to be generalizable.

In [56]:
%%time
min_l = 3
max_l = 20

X_train['lemmas_less'] = X_train['lemmas'].apply(lambda row: [lemma for lemma in row if len(lemma) >= min_l or len(lemma) <= max_l])

CPU times: user 781 ms, sys: 2.12 s, total: 2.9 s
Wall time: 10.5 s


# TF*IDF out of the Box

Resources:
- [TfidfVectorizer Docs](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)  

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [35]:
%%time
tfidf_sklearn = TfidfVectorizer(ngram_range = (1,3),
                                min_df = 1)

# return sparse matrix
# join list into individual strings
tfidf_values = tfidf_sklearn.fit_transform(X_train['lemmas_less'].apply(lambda x: " ".join(x)))

CPU times: user 33.4 s, sys: 1.17 s, total: 34.6 s
Wall time: 34.9 s


In [36]:
tfidf_values

<106912x4089577 sparse matrix of type '<class 'numpy.float64'>'
	with 8002628 stored elements in Compressed Sparse Row format>

In [37]:
%%time
'''
CPU times: user 11.9 s, sys: 8.27 s, total: 20.2 s
Wall time: 23 s
'''
lemmas_less_tfidf = pd.DataFrame(tfidf_values.toarray(),
                                 columns=tfidf_sklearn.get_feature_names())

CPU times: user 12.6 s, sys: 9.45 s, total: 22.1 s
Wall time: 25.6 s


In [38]:
lemmas_less_tfidf.shape

(106912, 4089577)

In [40]:
y_train['toxic'].shape

(106912,)

In [40]:
# sum(lemmas_less_tfidf['jerk'])

# search_term = 'jerk'

# # running this portion will crash the kernel
# bool_mask = lemmas_less_tfidf.sort_values(search_term, ascending=False)[search_term][:10]
# bool_mask

24.505664053836636

# Toxic: Random Forest Classifier

Resources:
- [Explanation of Warm Start for RFC (not what you may think)](https://stackoverflow.com/questions/42757892/how-to-use-warm-start/42763502)  

In [41]:
from sklearn.ensemble import RandomForestClassifier

In [46]:
toxic_rfc = RandomForestClassifier(n_estimators=100,
                                   max_depth = 10,
                                   oob_score=True,
                                   n_jobs=-1,
                                   random_state=42,
                                   warm_start=True)

In [47]:
%%time
X = lemmas_less_tfidf
y_toxic = y_train['toxic']

CPU times: user 12.5 ms, sys: 80.4 ms, total: 92.9 ms
Wall time: 31.8 ms


In [48]:
%%time
toxic_rfc.fit(X.iloc[:1000], y_toxic.iloc[:1000])

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:545: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


CPU times: user 2min 33s, sys: 7min 15s, total: 9min 49s
Wall time: 2min 55s


RandomForestClassifier(max_depth=10, n_estimators=1, n_jobs=-1, oob_score=True,
                       random_state=42, warm_start=True)

In [ ]:
# 

In [ ]:
# naive_toxic = BernoulliNB
# print(vect_X_train.shape)
# print(y_train.shape)
# print(X_train.shape)

# vect_X_train.iloc[0]

# naive_toxic.fit(vect_X_train, y_train)